# Train a Simplicial Neural Network for Homology Localization (Dist2Cycle)

In this notebook, we will create and train a Simplicial Neural Network for Homology Localization, as proposed in the paper by [Alexandros D. Keros et. al : Dist2Cycle: A Simplicial Neural Network for Homology Localization(2022)](https://ojs.aaai.org/index.php/AAAI/article/view/20673/20432). 

We train the model to perform binary node classification using the KarateClub benchmark dataset. 

The equations of one layer of this neural network are given by:

🟥 $\quad m^{(1 \rightarrow 1)}\_{y \rightarrow x}  = (A \odot (I + L\downarrow)^+{xy}) \cdot h_{y}^{t,(1)}\cdot \Theta^t$

🟧 $\quad m_x^{(1 \rightarrow 1)}  = \sum_{y \in \mathcal{L}\_\downarrow(x)} m_{y \rightarrow x}^{(1 \rightarrow 1)}$

🟩 $\quad m_x^{(1)}  = m^{(1 \rightarrow 1)}_x$

🟦 $\quad h_x^{t+1,(1)} = \sigma(m_{x}^{(1)})$



Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

In [1]:
import torch
import numpy as np

import toponetx.datasets.graph as graph

from topomodelx.nn.simplicial.dist2cycle import Dist2Cycle
from topomodelx.utils.sparse import from_sparse
import numpy.linalg as npla

%load_ext autoreload
%autoreload 2

# Pre-processing

## Import dataset ##

The first step is to import the Karate Club (https://www.jstor.org/stable/3629752) dataset. This is a singular graph with 34 nodes that belong to two different social groups. We will use these groups for the task of node-level binary classification.

We must first lift our graph dataset into the simplicial complex domain.

In [2]:
dataset = graph.karate_club(complex_type="simplicial")
print(dataset)

Simplicial Complex with shape (34, 78, 45, 11, 2) and dimension 4


## Define neighborhood structures. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on the domain. In this case, we need the boundary matrix (or incidence matrix) $B_1$ and the adjacency matrix $A_{\uparrow,0}$ on the nodes. For a santiy check, we show that the shape of the $B_1 = n_\text{nodes} \times n_\text{edges}$ and $A_{\uparrow,0} = n_\text{nodes} \times n_\text{nodes}$. We also convert the neighborhood structures to torch tensors.

In [3]:
incidence_1 = dataset.incidence_matrix(rank=1)
adjacency_0 = dataset.adjacency_matrix(rank=0)

incidence_1 = from_sparse(incidence_1)
adjacency_0 = from_sparse(adjacency_0)

print(f"The incidence matrix B1 has shape: {incidence_1.shape}.")
print(f"The adjacency matrix A0 has shape: {adjacency_0.shape}.")

The incidence matrix B1 has shape: torch.Size([34, 78]).
The adjacency matrix A0 has shape: torch.Size([34, 34]).


## Import signal ##

Since our task will be node classification, we must retrieve an input signal on the nodes. The signal will have shape $n_\text{nodes} \times$ in_channels, where in_channels is the dimension of each cell's feature. Here, we have in_channels = channels_nodes $ = 34$. This is because the Karate dataset encodes the identity of each of the 34 nodes as a one hot encoder.

In [4]:
x_0 = []
for _, v in dataset.get_simplex_attributes("node_feat").items():
    x_0.append(v)
x_0 = torch.tensor(np.stack(x_0))
channels_nodes = x_0.shape[-1]
print(channels_nodes)

2


In [5]:
print(f"There are {x_0.shape[0]} nodes with features of dimension {x_0.shape[1]}.")

There are 34 nodes with features of dimension 2.


To load edge features, this is how we would do it (note that we will not use these features for this model, and this serves simply as a demonstration).

In [6]:
x_1 = []
for k, v in dataset.get_simplex_attributes("edge_feat").items():
    x_1.append(v)
x_1 = np.stack(x_1)

In [7]:
print(f"There are {x_1.shape[0]} edges with features of dimension {x_1.shape[1]}.")

There are 78 edges with features of dimension 2.


Similarly for face features:

In [8]:
x_2 = []
for k, v in dataset.get_simplex_attributes("face_feat").items():
    x_2.append(v)
x_2 = np.stack(x_2)

In [9]:
print(f"There are {x_2.shape[0]} faces with features of dimension {x_2.shape[1]}.")

There are 45 faces with features of dimension 2.


## Define binary labels
We retrieve the labels associated to the nodes of each input simplex. In the KarateClub dataset, two social groups emerge. So we assign binary labels to the nodes indicating of which group they are a part.

We convert the binary labels into one-hot encoder form, and keep the first four nodes' true labels for the purpose of testing.

In [10]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)
y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y
y_train = y_true[:30]
y_test = y_true[-4:]

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

print(y_train.shape)

torch.Size([30, 2])


# Create Features

In [11]:
test = dataset.get_simplex_attributes("edge_feat")

ld = dataset.down_laplacian_matrix(rank=1).todense()
A = dataset.adjacency_matrix(rank=1).todense()
L_tilde_pinv = npla.pinv(ld + np.eye(ld.shape[0]))  # test inverse
channels_nodes = 78  # L_tilde_pinv.shape[-1]
print(channels_nodes)
print(np.array(A).shape)
print(np.array(ld).shape)
print(x_1.shape)  # edge features
print(L_tilde_pinv.shape)

adjacency = torch.from_numpy(A).float().to_sparse()
Linv = torch.from_numpy(L_tilde_pinv).float().to_sparse()

res = adjacency * Linv
print(res)
print(x_1)

x_1e = res.to_sparse()

incidence_0_1 = from_sparse(dataset.incidence_matrix(1))

78
(78, 78)
(78, 78)
(78, 2)
(78, 78)
tensor(indices=tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,
                         2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  4,
                         5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  8,  8,
                         8,  8, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 13, 13,
                        14, 14, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17,
                        17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19,
                        19, 19, 20, 20, 21, 21, 22, 22, 24, 24, 24, 24, 24, 24,
                        24, 24, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 28, 28,
                        28, 28, 28, 28, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33,
                        34, 34, 34, 34, 34, 34, 35, 35, 36, 36, 37, 37, 37, 37,
                        38, 38, 39, 39, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42,
  

/Users/gbg141/Documents/TopoProjectX/TopoModelX/venv_modelx/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


# Create the Neural Network

Using the SAN class, we create our neural network with stacked layers. Given the considered dataset and task (Karate Club, node classification), a linear layer at the end produces an output with shape $n_\text{nodes} \times 2$, so we can compare with our binary labels.

In [15]:
class Network(torch.nn.Module):
    def __init__(self, channels, out_channels, n_layers=2):
        super(Network, self).__init__()
        self.base_model = Dist2Cycle(
            channels=channels,
            n_layers=n_layers,
        )
        self.linear = torch.nn.Linear(channels, out_channels)

    def forward(self, x, incidence_1, adjacency_0):
        x = self.base_model(x, incidence_1, adjacency_0)
        x = self.linear(x)
        return torch.softmax(x, dim=1)

In [22]:
out_channels = 2
n_layers = 1

model = Network(
    channels=channels_nodes,
    out_channels=out_channels,
    n_layers=3,
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

The following cell performs the training, looping over the network for a low number of epochs.

In [23]:
test_interval = 10
num_epochs = 100
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()

    y_hat_edge = model(x_1e, Linv, adjacency)
    # We project the edge-level output of the model to the node-level
    # and apply softmax fn to get the final node-level classification output
    y_hat = torch.softmax(torch.sparse.mm(incidence_0_1, y_hat_edge), dim=1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        y_hat[: len(y_train)].float(), y_train.float()
    )
    epoch_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.where(y_hat > 0.5, torch.tensor(1), torch.tensor(0))
    accuracy = (y_pred[: len(y_train)] == y_train).all(dim=1).float().mean().item()
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {accuracy:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            y_hat_edge_test = model(x_1e, Linv, adjacency)
            # Projection to node-level
            y_hat_test = torch.softmax(
                torch.sparse.mm(incidence_0_1, y_hat_edge_test), dim=1
            )
            y_pred_test = torch.where(
                y_hat_test > 0.5, torch.tensor(1), torch.tensor(0)
            )
            # _pred_test = torch.softmax(y_hat_test,dim=1).ge(0.5).float()
            test_accuracy = (
                torch.eq(y_pred_test[-len(y_test) :], y_test)
                .all(dim=1)
                .float()
                .mean()
                .item()
            )
            print(f"Test_acc: {test_accuracy:.4f}", flush=True)

Epoch: 1 loss: 0.7234 Train_acc: 0.6333
Epoch: 2 loss: 0.6930 Train_acc: 0.6667
Epoch: 3 loss: 0.6769 Train_acc: 0.7000
Epoch: 4 loss: 0.6718 Train_acc: 0.7000
Epoch: 5 loss: 0.6701 Train_acc: 0.7000
Epoch: 6 loss: 0.6694 Train_acc: 0.7000


Epoch: 7 loss: 0.6691 Train_acc: 0.7000
Epoch: 8 loss: 0.6689 Train_acc: 0.7000
Epoch: 9 loss: 0.6689 Train_acc: 0.7000
Epoch: 10 loss: 0.6688 Train_acc: 0.7000
Test_acc: 0.2500
Epoch: 11 loss: 0.6688 Train_acc: 0.7000
Epoch: 12 loss: 0.6688 Train_acc: 0.7000
Epoch: 13 loss: 0.6688 Train_acc: 0.7000
Epoch: 14 loss: 0.6688 Train_acc: 0.7000
Epoch: 15 loss: 0.6688 Train_acc: 0.7000
Epoch: 16 loss: 0.6688 Train_acc: 0.7000
Epoch: 17 loss: 0.6688 Train_acc: 0.7000
Epoch: 18 loss: 0.6688 Train_acc: 0.7000
Epoch: 19 loss: 0.6688 Train_acc: 0.7000
Epoch: 20 loss: 0.6688 Train_acc: 0.7000
Test_acc: 0.2500
Epoch: 21 loss: 0.6688 Train_acc: 0.7000
Epoch: 22 loss: 0.6688 Train_acc: 0.7000
Epoch: 23 loss: 0.6688 Train_acc: 0.7000
Epoch: 24 loss: 0.6688 Train_acc: 0.7000
Epoch: 25 loss: 0.6688 Train_acc: 0.7000
Epoch: 26 loss: 0.6688 Train_acc: 0.7000
Epoch: 27 loss: 0.6688 Train_acc: 0.7000
Epoch: 28 loss: 0.6688 Train_acc: 0.7000
Epoch: 29 loss: 0.6688 Train_acc: 0.7000
Epoch: 30 loss: 0.6688 Tra